# 项目背景与方法 

## 背景描述
近年来，随着新零售业的快速发展，消费者购买商品时有了更多的对比和选择，导致超市行业的竞争日益激烈，利润空间不断压缩。超市的经营管理产生了大量数据，对这些数据进行分析，可以提升超市的竞争力，为超市的运营及经营策略调整提供重要依据。

## 数据来源及介绍
ModelWhale平台上泰迪杯”数据分析大赛A题 超市销售数据分析。为某超市 2015 年 1 月 1 日至 4 月 30 的经营数据。

| 列名     | 说明    | 
| -------  | -------|
| 顾客编码  | 整数类型  |  
| 大类编码  | 整数类型  | 
| 大类名称  | 字符串    |  
| 中类编码  | 整数类型  |
| 中类名称  | 字符串    |
| 小类编码  | 整数类型  |
| 小类名称  | 字符串    |
| 中类编码  | 整数类型  |
| 销售日期  | 时间类型  |
| 销售月份  | 时间类型  |
| 商品编码  | 字符串    |
| 规格型号  | 字符串    |
| 商品类型  | 字符串    |
| 单位     | 字符串    |
| 销售数量  | 整数类型  |
| 销售金额  | 浮点类型  |
| 商品单价  | 浮点类型  |
| 是否促销  | 否       |


## 问题描述
* 对销售数据进行统计分析，并作可视化展示。
* 分析顾客的消费行为。
* 研究促销对销售的影响。

## 技术方法
使用图形界面工具Navicat完成数据导入，清洗
使用python中的matplotlib完成可视化？

# 数据导入
 

In [ ]:
#关键过程如下

![图片](图片/1.png  "新建数据库")
新建数据库

![图片](图片/2.png)
数据导入

![图片](图片/3.png)
日期格式设置

![图片](图片/4.png)
表格重命名

![图片](图片/5.png)
字段类型设置

![图片](图片/6.png)
 成功将全部42816条数据导入

In [ ]:
销售日期和销售月份字段信息重复，删除销售月份列
ALTER TABLE salesdata DROP COLUMN 销售月份

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import plotly
import plotly.graph_objects as go
from IPython.display import HTML

#解决中文字体不显示，正负号不显示问题
plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

%matplotlib widget

warnings.filterwarnings('ignore')

#  数据清洗

## 删除重复数据

In [ ]:
从查询结果可知，数据中有3重复记录。

In [ ]:
#删除重复数据
CREATE table salesdatas SELECT DISTINCT * FROM salesdata;
DROP TABLE salesdata
ALTER TABLE salesdatas RENAME TO salesdata

In [ ]:
#得到42813条无重复记录
SELECT count(*) FROM salesdata

## 缺失值查询与处理

![图片](图片/7.png)
各字段缺失值统计

In [ ]:
从查询结果来看，销售数量字段有两个缺失值

In [ ]:
删除销售字段为空的记录
DELETE FROM salesdata WHERE 销售数量 is null

In [ ]:
再次查看是否有缺失值
SELECT COUNT(顾客编号),COUNT(大类编码),COUNT(大类名称),COUNT(中类编码),COUNT(中类名称),COUNT(小类编码),
count(小类名称),count(销售日期),count(商品编码),count(规格型号),count(商品类型),count(单位),
count(销售数量),count(销售金额),count(商品单价),count(是否促销)
FROM salesdata

In [ ]:
从查询结果可以看出，各字段已没有缺失值

## 异常值查询与处理

![图片](图片/8.png)
从查询结果可以看出，销售金额和销售数量有负值

![图片](图片/9.png)

![图片](图片/10.png)

## 日期数据格式转换

![图片](图片/11.png)

![图片](图片/12.png)

# 基于商品维度分析

## 日销售规律探索
每日顾客总数
每日销售总数
每日销售金额总数

注意：一天同一个客户的所有购买记录算作一个购物篮（一个销售单）
SELECT 销售日期 ,count(distinct 顾客编号) 顾客人数,
    sum(销售数量) 商品销量,sum(销售金额) 销售金额
FROM salesdata
GROUP BY 销售日期

In [ ]:
##日销售规律
SELECT 销售日期 ,count(distinct 顾客编号) 顾客人数,
    sum(销售数量) 商品销量,sum(销售金额) 销售金额
FROM salesdata
GROUP BY 销售日期

In [ ]:
# df = pd.read_clipboard()
# df.to_csv('./data1.csv',index=False)

In [ ]:
data1 = pd.read_csv('./data1.csv').set_index('销售日期')

In [ ]:
data1

In [ ]:
def figure1(data):
    fig = plt.figure(figsize=(8,6))
    axe = fig.add_subplot(111)
    axe.set_title('日销售规律图')
    axe.set_ylabel('数量')
    axe.plot(data.index,data['销售金额'],'b-',alpha=0.7,label='销售金额')
    axe.plot(data.index,data['商品销量'],'m-',alpha=0.7,label='商品销量')  
    plt.legend()
    axe1 = axe.twinx()
    axe1.set_ylabel('顾客人数')
    axe1.plot(data.index,data['顾客人数'],'r-',alpha=0.7,label='顾客人数') 
    plt.legend(loc=7)
    plt.xticks(range(1,116)[::15],data.index.to_list()[::15])

In [ ]:
figure1(data1)


![图片](图片/13.png)

异常数据为2015年2月4日，当天周三，不是法定假日，却只有一个顾客

## 周销售规律探索

In [ ]:
#购物篮系数周变化趋势
#从日销售规律可以看出，节假日对销量影响比较大，为了探究日常每周销售规律，需剔除节假日的影响

SELECT dayofweek(销售日期) as 周,sum(顾客人数) 顾客人数,
    sum(商品销量) 商品销量,sum(销售金额) 销售金额
FROM(
		SELECT 销售日期 ,count(distinct 顾客编号) 顾客人数,
				sum(销售数量) 商品销量,sum(销售金额) 销售金额
		FROM salesdata
		GROUP BY 销售日期
		) as a
WHERE 销售日期 not in ('2015-1-1','2015-2-11','2015-2-12','2015-2-13','2015-2-14','2015-2-15','2015-2-16'
                   ,'2015-2-17','2015-2-18','2015-2-19','2015-2-20','2015-2-21','2015-2-22','2015-2-23'
                   ,'2015-2-24','2015-3-5','2015-3-8','2015-4-5')
GROUP BY 周;

In [ ]:
# df = pd.read_clipboard()
# df.to_csv('./data2.csv',index=False)
data2 = pd.read_csv('./data2.csv').set_index('星期')

In [ ]:
data2
# data2.index

In [ ]:
def figure(data):
    fig = plt.figure()
    axe = fig.add_subplot(111)
    axe.set_title('周销售规律图')
    axe.set_ylabel('数量')
    axe.plot(data.index,data['销售金额'],'b-',alpha=0.7,label='销售金额')
    axe.plot(data.index,data['商品销量'],'m-',alpha=0.7,label='商品销量')  
    plt.legend()
    axe1 = axe.twinx()
    axe1.set_ylabel('销售单数量')
    axe1.plot(data.index,data['顾客人数'],'r-',alpha=0.7,label='销售单数量') 
    plt.legend(loc=7)
    # plt.xticks(range(1,116)[::15],data.index.to_list()[::15])


In [ ]:
figure(data2)

In [ ]:
#周三的销量最好，可用来进行促销活动，或者设定会员日
#周四相对最差，可以挑选销售最差的周四送货、盘点货架等，也可通过设定会员日、促销等手段改善销售业绩

## 贡献度分析
累计贡献率（%）=（累计销售收入/销售总收入）*100%

In [ ]:
SELECT 大类名称,销售金额, @累计金额:=(销售金额+@累计金额), round(@累计金额/(SELECT sum(销售金额) FROM salesdata),2) 累计贡献率
FROM (
		SELECT 大类名称,sum(销售金额) 销售金额
		FROM salesdata
		GROUP BY 大类名称
		ORDER BY 销售金额 DESC
) as a ,(SELECT @累计金额:=0) as b

![图片](图片/23.png)

In [ ]:
# df = pd.read_clipboard()
# df.to_csv('./data5.csv',index=False)
data5 = pd.read_csv('./data5.csv')

In [ ]:
data5

In [ ]:
line = go.Scatter(x=data5['大类名称'],y=data5['累计贡献率'],name='累计贡献率',yaxis='y2',
                  marker=dict(size=8,color='blue'))
bar = go.Bar(x=data5['大类名称'],y=data5['销售金额'],text=data5['销售金额'],
             textposition='outside',name='销售金额',marker_color='rgb(200, 18, 100)')
layout = go.Layout(title = '产品贡献度分析',title_x=0.5,
                   xaxis_title = '大类名称',yaxis_title = '销售金额',yaxis2=dict(title='累计贡献率',overlaying='y',side='right'))

fig = go.Figure([line,bar],layout=layout)
fig.add_annotation(text="78%",x="酒饮", y=61213, arrowhead=1, showarrow=True,arrowcolor='red')
HTML(fig.to_html())

## 人气商品分析
指定商品购物篮系数=某段时间含指定商品购物篮的销售总数/某段时间含指定商品购物篮总数

指定商品的人气指数=某段时间含指定商品购物篮的销售总数/某段时间购物篮总数

购物篮系数：顾客平均购买数量

销售单总量（购物篮数量）：每日同一个客户购买的所有商品为一个销售单

In [ ]:
#从大类的角度来分析人气类别
#大类的总数：15
SELECT count(distinct 大类编码 ),count(DISTINCT 大类名称)
FROM salesdata 

![图片](图片/14.png)

In [ ]:
# df = pd.read_clipboard()
# df.to_csv('./data3.csv',index=False)
data3 = pd.read_csv('./data3.csv')

In [ ]:
data3

In [ ]:
def figure3(data):
    fig = plt.figure()
    axe = fig.add_subplot(111)
    axe.set_title('大类的购物篮数量-购物篮系数四象限图')
    axe.set_ylabel('含指定大类的销售单数')
    axe.set_xlabel('购物篮系数')
    axe.scatter(data['购物篮系数'],data['含指定大类的销售单数']) 
    plt.axhline(y=data['含指定大类的销售单数'].mean(),ls="-",color="#d62728",linewidth=1)#y=0表示水平线过y=0，ls设置线条类型，linewidth设置线条粗细
    plt.axvline(x=data['购物篮系数'].mean(),ls="-",color="#d62728",linewidth=1)
    plt.annotate('红线为均值线',xy=(1,50),xytext=(1.2,50),xycoords='data',
               )
    

In [ ]:
figure3(data3)

In [ ]:
# 第一象限（右上角）：购物篮系数和购物篮数量都比较高，属于人气商品
# 第二象限（左上角）：购物篮数量比较高，但购物篮系数低于平均值，这部分商品需要解决的是如果提高它们的关联销售
# 第三象限（左下角）：购物篮系数和购物篮数量都比较低，基本属于边缘商品，本身卖的不好也与其他商品关联度不高
# 第四象限（右下角）：购物篮系数比较高，购物篮数量低于平均值，这部分商品需要解决的是如果提高它们的购物篮数量

In [ ]:
#处于第一象限的大类名单:粮油,日配
SELECT 大类名称 
FROM view1
WHERE 含指定大类的销售单数>=(SELECT avg(含指定大类的销售单数) FROM view1)
AND 购物篮系数>=(SELECT avg(购物篮系数) FROM view1)
#处于第二象限的大类名单:蔬果,休闲,洗化
SELECT 大类名称 
FROM view1
WHERE 含指定大类的销售单数>=(SELECT avg(含指定大类的销售单数) FROM view1)
AND 购物篮系数<(SELECT avg(购物篮系数) FROM view1)
#处于第三象限的大类名单:肉禽,熟食,水产,家电,烘焙
SELECT 大类名称 
FROM view1
WHERE 含指定大类的销售单数<(SELECT avg(含指定大类的销售单数) FROM view1)
AND 购物篮系数<(SELECT avg(购物篮系数) FROM view1)
#处于第四象限的大类名单:酒饮,冲调,家居,针织,文体
SELECT 大类名称 
FROM view1
WHERE 含指定大类的销售单数<(SELECT avg(含指定大类的销售单数) FROM view1)
AND 购物篮系数>=(SELECT avg(购物篮系数) FROM view1)

In [ ]:
#指定商品人气指数=某段时间含指定商品购物篮的销售总数/某段时间的购物篮总数
#挑选销量前5的大类分析其人气指数
CREATE VIEW view2
AS
SELECT *
FROM (
		SELECT 大类名称,销售数量,row_number()over(ORDER BY 销售数量 DESC) as rank1
		FROM 
		(
				SELECT 大类名称,sum(销售数量) 销售数量
				FROM salesdata
				GROUP BY 大类名称
		) as a
) as tempp
WHERE rank1<=5

SELECT view1.大类名称,购物篮系数,人气指数,销售数量
FROM view1
JOIN view2
ON view1.大类名称=view2.大类名称


![图片](图片/15.png)

## 商品的关联程度分析
品类间的关联
* 选择销量前5的大类进行关联度分析:日配 蔬果 粮油 休闲 酒饮

In [ ]:
#支持度、可信度、提升度
# 支持度：同时包含A和B的交易/总交易*100%
# 可信度：同时包含A和B的交易/包含A的总交易*100%
# 提升度：可信度/商品B在总交易中出现的概率



In [ ]:
#创建临时表view3，计算销量前5的商品大类关联交易数量

![图片](图片/16.png)

In [ ]:
#创建临时表view4，计算销量前5的商品大类在总交易中出现的的概率

In [ ]:
CREATE VIEW view4
AS
SELECT 商品大类,(case when  商品大类='蔬果' then 同时购买蔬果/总交易数
					when 商品大类='日配' then 同时购买日配/总交易数
					when 商品大类='粮油' then 同时购买粮油/总交易数
					when 商品大类='休闲' then 同时购买休闲/总交易数
					when 商品大类='酒饮' then  同时购买酒饮/总交易数 
					end ) 各商品在总交易中出现的概率
FROM view3;

![图片](图片/17.png)

In [ ]:
#支持度表

In [ ]:
SELECT 商品大类,同时购买蔬果/总交易数 同时购买蔬果,同时购买日配/总交易数 同时购买日配 ,同时购买粮油/总交易数 同时购买粮油,
同时购买休闲/总交易数 同时购买休闲, 同时购买酒饮/总交易数 同时购买酒饮
FROM view3

![图片](图片/18.png)

In [ ]:
#可信度表

In [ ]:
CREATE VIEW kexingdu
AS
SELECT 商品大类,
	(case when 商品大类='蔬果' then 1
	      when 商品大类='日配' then 同时购买蔬果/同时购买日配
				when 商品大类='粮油' then 同时购买蔬果/同时购买粮油
				when 商品大类='休闲' then 同时购买蔬果/同时购买休闲
				when 商品大类='酒饮' then 同时购买蔬果/同时购买酒饮
end ) 同时购买蔬果,
(case when 商品大类='日配' then 1 
	      when 商品大类='蔬果' then 同时购买日配/同时购买蔬果
				when 商品大类='粮油' then 同时购买日配/同时购买粮油
				when 商品大类='休闲' then 同时购买日配/同时购买休闲
				when 商品大类='酒饮' then 同时购买日配/同时购买酒饮
end ) 同时购买日配,
(case when 商品大类='粮油' then 1
	      when 商品大类='蔬果' then 同时购买粮油/同时购买蔬果
				when 商品大类='日配' then 同时购买粮油/同时购买日配
				when 商品大类='休闲' then 同时购买粮油/同时购买休闲
				when 商品大类='酒饮' then 同时购买粮油/同时购买酒饮
end ) 同时购买粮油,
(case when 商品大类='休闲' then 1 
	      when 商品大类='蔬果' then 同时购买休闲/同时购买蔬果
				when 商品大类='粮油' then 同时购买休闲/同时购买粮油
				when 商品大类='日配' then 同时购买休闲/同时购买日配
				when 商品大类='酒饮' then 同时购买休闲/同时购买酒饮
end ) 同时购买休闲,
(case when 商品大类='酒饮' then 1 
	      when 商品大类='蔬果' then 同时购买酒饮/同时购买蔬果
				when 商品大类='粮油' then 同时购买酒饮/同时购买粮油
				when 商品大类='休闲' then 同时购买酒饮/同时购买休闲
				when 商品大类='日配' then 同时购买酒饮/同时购买日配
end ) 同时购买酒饮
FROM view3;

![图片](图片/19.png)

In [ ]:
#提升度表

![图片](图片/20.png)

In [ ]:
#综合支持度、可信度、提升度，销量前五的大类中酒饮和休闲的关联性强，可考虑捆绑销售，或者在货架陈列时放在一起

# RFM模型分析
R：客户最近一次消费距离当前日期天数（当前日期为数据中的最晚日期）
F：客户在某段时间的消费频率
M：客户在某段时间的消费总金额

In [ ]:
#创建RFM表

In [ ]:
CREATE VIEW RFMtable
AS
SELECT 顾客编号,TIMESTAMPDIFF(day,max(日期),(SELECT max(销售日期) FROM salesdata)) R,
count(日期) F,SUM(M) M
FROM(
		SELECT `顾客编号`,`销售日期`,max(销售日期) 日期,sum(`销售金额`) M
		FROM salesdata
		GROUP BY `顾客编号`,`销售日期`
		ORDER BY `顾客编号`,`销售日期`
) as a 
GROUP BY 顾客编号

In [ ]:
SELECT label ,count(顾客编号) 人数,round(std(R),2) 平均R,round(std(F),2) 平均F,round(std(M),2) 平均M
FROM (
		SELECT *,(case when concat(r_s,f_s,m_s)='111' then '一般挽留客户'
									 when concat(r_s,f_s,m_s)='112' then '重要挽留客户'
									 when concat(r_s,f_s,m_s)='121' then '一般保持客户'
									 when concat(r_s,f_s,m_s)='122' then '重要保持客户'
									 when concat(r_s,f_s,m_s)='211' then '一般发展客户'
									 when concat(r_s,f_s,m_s)='212' then '重要发展客户'
									 when concat(r_s,f_s,m_s)='221' then '一般价值客户'
									 when concat(r_s,f_s,m_s)='222' then '重要价值客户'
						end ) label
		FROM(
				SELECT *,( case when R>=(SELECT std(R) FROM RFMtable) THEN 1 else 2 END) r_s,
				( case when F>=(SELECT std(F) FROM RFMtable) THEN 2 else 1 END) f_s,
				( case when M>=(SELECT std(M) FROM RFMtable) THEN 2 else 1 END) m_s
				FROM RFMtable
		) as a 
) as b
GROUP BY label


![图片](图片/21.png)

In [ ]:
#对不同价值用户采用不同的营销策略 ，比如：
#重要价值客户：提供高品质的会员服务 
#一般挽留客户：增加推送，激活用户

# 促销分析

In [ ]:
#每次购物是否购买促销品来对客户进行分类
CREATE VIEW cuxiao_kehu
AS
SELECT 顾客编号,消费次数,购买过促销消费占比,
	 (case when 购买过促销消费占比=0 then '不喜欢促销品'
	 when 购买过促销消费占比>0 and 购买过促销消费占比<0.5  then '喜欢促销品'
	 when 购买过促销消费占比>=0.5  then '非常喜欢促销品'
	 end ) 喜欢促销品的程度
FROM(
		SELECT 顾客编号,count(销售日期) 消费次数 ,round(sum(case when 是否购买促销<>0 then 1 else 0 end )/count(销售日期),2) 购买过促销消费占比
		FROM (
				SELECT`顾客编号`,`销售日期`,
				sum(case when 是否促销='是' then 1 else 0 end) 是否购买促销
				FROM salesdata
				GROUP BY `顾客编号`,`销售日期`
				ORDER BY  `顾客编号`,`销售日期`
		) as a 
		GROUP BY 顾客编号
) as b

In [ ]:
#统计对促销喜欢程度不同的客户数量

In [ ]:
SELECT 喜欢促销品的程度,count(顾客编号)
FROM cuxiao_kehu
GROUP BY  喜欢促销品的程度

![图片](图片/22.png)

In [ ]:
# 每天曾参加促销的小类占比与整体的销售金额的关系
SELECT 销售日期,round(sum(case when 是否促销='是' then 1 else 0 end )/count(小类名称),2) 每日促销的小类种类占比,sum(销售金额) 销售金额
FROM(
		SELECT 销售日期,小类名称,是否促销,sum(销售金额) 销售金额
		FROM salesdata
		GROUP BY 销售日期,小类名称,是否促销
) as a
GROUP BY 销售日期

In [ ]:
# df = pd.read_clipboard()
# df.to_csv('./data4.csv',index=False)

In [ ]:
data4 = pd.read_csv('./data4.csv').set_index('销售日期')

In [ ]:
data4

In [ ]:
fig = plt.figure()
axe = fig.add_subplot(111)
axe.plot(data4.index,data4['销售金额'],label='销售金额')
axe.set_ylabel('销售金额')
plt.legend(loc=3)
axe1 = axe.twinx()
axe1.set_ylabel('每日促销的小类种类占比')
axe1.plot(data4.index,data4['每日促销的小类种类占比'],'r--',label='每日促销的小类种类占比')
plt.xticks(data4.index[::15])
plt.legend()

In [ ]:
#销售金额在一定程度上与促销有=关